In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline
from fish.ephys.ephys import load, estimate_onset, chop_trials
from triggered_average import trigger_data, match_cam_time, load_single_plane
from fish.image.vol import get_stack_dims
from fish.util.fileio import load_image

In [2]:
dirs = {}
dirs['ephys'] = r'R:/davis/data/ephys/20171108/7dpf_cy171xec43_f1_opto_2_12mw.10chFlt'
dirs['ims'] = r'F:/davis/20171108/7dpf_cy171xec43_f1_opto_2_12mw_20171108_215904/'

dims = get_stack_dims(dirs['ims'])[::-1]
single_plane = False
if dims[0] == 50:
    single_plane = True

fnames = sorted(glob(dirs['ims'] + 'TM*'))
# Number of timepoints per file is 50 for high speed single plane
num_frames = len(fnames)
if single_plane:
    num_frames = len(fnames) * 50
print(len(fnames))
print(num_frames)

3890
3890


In [3]:
ep_dat = load(dirs['ephys'])
cam_times = estimate_onset(ep_dat[2], threshold=3, duration=50)[:num_frames]
laser_on = estimate_onset(ep_dat[3], threshold=3.7, duration=400)
laser_off = len(ep_dat[3]) - estimate_onset(ep_dat[3][::-1], threshold=3.7, duration=400)
print(len(cam_times))

3890


In [4]:
trials = chop_trials(ep_dat[4])
# for each trial onset look for the nearest cam time before the onset.
# for each offset, look for the nearest cam time after offset
trials_cam = {}

for key, val in trials.items():
    pre = match_cam_time(val[0], cam_times, timing='pre')
    post = match_cam_time(val[1], cam_times, timing='post')
    trials_cam[key] = [pre, post]

In [7]:
# Get all events
window=np.arange(-5, 10)
triggered = {}
for key, value in trials_cam.items():
    if key > 0:
        print(key)
        
        # don't trigger on values where we can't get a full window
        trigs = list(filter(lambda v: v + window[-1] < num_frames, value[-1]))
        print(trigs)
        triggered[key] = trigger_data(trigs[::5], window, fnames, average=True, z=None)

1.0
[32, 275, 373, 567, 616, 810, 907, 1053, 1248, 1393, 1491, 1685, 1831, 1928, 2171, 2269, 2463, 2512, 2755, 2852, 3046, 3095, 3338, 3387, 3630, 3727]
2.0
[81, 178, 324, 518, 664, 859, 956, 1151, 1199, 1442, 1539, 1734, 1782, 2026, 2123, 2317, 2414, 2609, 2658, 2901, 2998, 3144, 3241, 3435, 3581, 3678, 3824]
3.0
[130, 227, 421, 470, 713, 761, 1005, 1102, 1296, 1345, 1588, 1637, 1880, 1977, 2074, 2220, 2366, 2560, 2706, 2803, 2949, 3192, 3290, 3484, 3533, 3776, 3873]


In [10]:
import pyqtgraph as pq

%gui qt

ImportError: cannot import name 'QtGui'

In [9]:
triggered[1.0].shape

(15, 45, 1024, 2048)

In [ ]:
plt.imshow(triggered[1.0].max((0,1)))

In [ ]:
z = 30
pq.image(triggered[1.0][:,z], title='roi 1.0')
pq.image(triggered[2.0][:,z], title='roi 2.0')
pq.image(triggered[3.0][:,z], title='roi 3.0')

In [ ]:
pq.image(triggered[1.0].max((0)))

In [ ]:
pq.image(triggered[1.0][:,-15])